## Intcode

In [73]:
# states are
# 0 - running
# 1 - waiting for input
# 99 - halt

mutable struct Prog
    code::Vector{Int}
    cur::Int
    input::Vector{Int}
    output::Vector{Int}
    relative_base::Int
    state::Int
end

str2prog(s) = parse.(Int, split(s, ","))

Prog(s::String) = Prog(vcat(str2prog(s), zeros(Int, 10000)), 1, [], [], 0, 0)

struct Instruction
    op::Int
    modes::Vector{Int}
end

function Instruction(op_code::Int)
    ops = Dict{Int, Int}(
        1 => 3,
        2 => 3,
        3 => 1,
        4 => 1,
        5 => 2,
        6 => 2,
        7 => 3,
        8 => 3,
        9 => 1,
        99 => 0
    )
    
    op = mod(op_code, 100)
    modes_code = div(op_code, 100)
    modes = zeros(ops[op])
    for i in 1:ops[op]
        modes[i] = mod(modes_code, 10)
        modes_code = div(modes_code, 10)
    end
    
    Instruction(op, modes)
end

Instruction(prog::Prog) = Instruction(prog.code[prog.cur])

function take(prog::Prog, mode, offset)
    if mode == 0
        return prog.code[prog.code[prog.cur + offset] + 1]
    elseif mode == 1
        return prog.code[prog.cur + offset]
    else
        return prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1]
    end
end

function update!(prog::Prog, value, offset, mode = 0)
    if mode == 0
        prog.code[prog.code[prog.cur + offset] + 1] = value
    elseif mode == 2
        prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1] = value
    end
end

function apply(prog::Prog, instruction::Instruction)
    if instruction.op == 99
        prog.state = 99
    elseif instruction.op == 1
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 + a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 2
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 * a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 3
        if isempty(prog.input)
            prog.state = 1
        else
            update!(prog, popfirst!(prog.input), 1, instruction.modes[1])
            prog.cur += 2
        end
    elseif instruction.op == 4
        push!(prog.output, take(prog, instruction.modes[1], 1))
        prog.cur += 2
    elseif instruction.op == 5
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 != 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 6
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 == 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 7
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 < a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 8
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 == a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 9
        prog.relative_base += take(prog, instruction.modes[1], 1)
        prog.cur += 2
    end
end

function run(prog::Prog, input::Vector{Int})
    prog.input = input
    prog.state = 0
    run(prog)
end

run(prog::Prog, input::Int) = run(prog, [input])

function run(prog)
    clear!(prog)
    while prog.state == 0
        instruction = Instruction(prog)
        apply(prog, instruction)
    end
    
    prog.output
end

function feed(prog::Prog, input::Vector{Int})
    prog.input = vcat(prog.input, input)
    prog.state = 0
end

clear!(prog) = (prog.output = [])

clear! (generic function with 1 method)

## Part 1

In [4]:
vm = Prog(readline("input.txt"))
img_txt = run(vm)

2497-element Array{Int64,1}:
 46
 46
 46
 46
 46
 46
 46
 46
 46
 46
 46
 46
 46
  ⋮
 46
 46
 46
 46
 46
 46
 46
 46
 46
 46
 10
 10

In [16]:
# 46 = '.'
# 35 = '#'
# 10 = '\n'
# 94 = '^'

# img_txt[(img_txt .!= 46) .& (img_txt .!= 35) .& (img_txt .!= 10)]
print(join(Char.(img_txt)))

..............................................###########......
..............................................#.........#......
..............................................#.........#......
..............................................#.........#......
......#######.................................#.........#......
......#.....#.................................#.........#......
......#.....#.........#.......................#.........#......
......#.....#.........#.......................#.........#......
......#.....#.........#.......................#.........#......
......#.....#.........#.......................#.........#......
......#.....#############...................#############......
......#...............#.#...................#.#................
......#...............#############.....#######................
......#.................#.........#.....#...#..................
......#.................#.........#.....#...#..................
......#.................#.........#.....

In [83]:
img_arr = argmax(img_txt .== 10) |> x -> reshape(img_txt[1:(end-1)], x, (length(img_txt) - 1) ÷ x)[1:(end - 1), :]
h, w = size(img_arr)
mask = reshape(2 .^ (0:8), 3, 3)
filtered = sum(map(x -> mask[x]*map(y -> y == 46 ? 0 : 1, @view img_arr[x[1]:(h + x[1] - 3), x[2]:(w + x[2] - 3)]), 
    CartesianIndices(mask))) .== 186
sum([x[1]*x[2]*filtered[x] for x in CartesianIndices(filtered)])

13580

In [94]:
padding = fill(46, h + 2, w + 2)
padding[2:(h + 1), 2:(w + 1)] .= img_arr
CartesianIndices(padding)[padding .== 94]

1-element Array{CartesianIndex{2},1}:
 CartesianIndex(2, 18)

In [154]:
mutable struct Droid
    pos::CartesianIndex{2}
    dir::Vector{Int}
end

rotations = Dict(
    'R' => [0 -1; 1 0],
    'M' => [1 0; 0 1],
    'L' => [0 1; -1 0]
)

Droid(pos) = Droid(pos, [0, -1])
rotation(droid::Droid, dir::Char) = rotations[dir] * droid.dir
rotate(droid::Droid, dir::Char) = droid.dir = rotations[dir] * droid.dir
move(droid::Droid) = droid.pos += CartesianIndex(droid.dir...)
sense(droid, arr) = map(x -> arr[droid.pos + CartesianIndex(rotation(droid, x)...)], ['L', 'M', 'R']) .!= 46

sense (generic function with 1 method)

In [133]:
droid = Droid(CartesianIndices(padding)[padding .== 94][1])

Droid(CartesianIndex(2, 18), [0, -1])

In [164]:
function construct_path(droid::Droid, arr)
    res = ""
    cnt = 0
    while true
        vars = sense(droid, arr)
        if vars[2]
            cnt += 1
            move(droid)
        else
            res *= "," * string(cnt)
            if sum(vars) == 0 return res[4:end] end
            cnt = 0
            res *= vars[1] ? ",L" : ",R"
            rotate(droid, vars[1] ? 'L' : 'R')
        end
    end
end

construct_path (generic function with 1 method)

In [167]:
droid = Droid(CartesianIndices(padding)[padding .== 94][1])
path = construct_path(droid, padding)

"R,6,L,12,R,6,R,6,L,12,R,6,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,6,L,12,R,6"

In [1]:
function gen_subroutine(path)
    init = "^(.*),(.*),(.*)"
    stack = [init]
    println("-----------")
    while true
        r = pop!(stack)
        println(r)
        stack = vcat(map(x -> r * x, [",\\1", ",\\2", ",\\3"]), stack)
        m = match(Regex(r * "\$"), path)
        if m != nothing
            if all(length.(m.captures) .<= 20) return string.(m.captures) end
        end
    end
end

gen_subroutine (generic function with 1 method)

In [2]:
gen_subroutine("R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2")

-----------
^(.*),(.*),(.*)
^(.*),(.*),(.*),\3
^(.*),(.*),(.*),\2
^(.*),(.*),(.*),\1
^(.*),(.*),(.*),\3,\3
^(.*),(.*),(.*),\3,\2
^(.*),(.*),(.*),\3,\1
^(.*),(.*),(.*),\2,\3
^(.*),(.*),(.*),\2,\2
^(.*),(.*),(.*),\2,\1
^(.*),(.*),(.*),\1,\3
^(.*),(.*),(.*),\1,\2
^(.*),(.*),(.*),\1,\1
^(.*),(.*),(.*),\3,\3,\3
^(.*),(.*),(.*),\3,\3,\2
^(.*),(.*),(.*),\3,\3,\1
^(.*),(.*),(.*),\3,\2,\3
^(.*),(.*),(.*),\3,\2,\2
^(.*),(.*),(.*),\3,\2,\1
^(.*),(.*),(.*),\3,\1,\3
^(.*),(.*),(.*),\3,\1,\2
^(.*),(.*),(.*),\3,\1,\1
^(.*),(.*),(.*),\2,\3,\3
^(.*),(.*),(.*),\2,\3,\2
^(.*),(.*),(.*),\2,\3,\1
^(.*),(.*),(.*),\2,\2,\3
^(.*),(.*),(.*),\2,\2,\2
^(.*),(.*),(.*),\2,\2,\1
^(.*),(.*),(.*),\2,\1,\3


3-element Array{String,1}:
 "R,8,R,8"    
 "R,4,R,4,R,8"
 "L,6,L,2"    

In [ ]:
gen_subroutine("R,6,L,12,R,6,R,6,L,12,R,6,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,6,L,12,R,6")

-----------
^(.*),(.*),(.*)
^(.*),(.*),(.*),\3
^(.*),(.*),(.*),\2
^(.*),(.*),(.*),\1
^(.*),(.*),(.*),\3,\3
^(.*),(.*),(.*),\3,\2
^(.*),(.*),(.*),\3,\1
^(.*),(.*),(.*),\2,\3
^(.*),(.*),(.*),\2,\2
^(.*),(.*),(.*),\2,\1
^(.*),(.*),(.*),\1,\3
^(.*),(.*),(.*),\1,\2
^(.*),(.*),(.*),\1,\1
^(.*),(.*),(.*),\3,\3,\3
^(.*),(.*),(.*),\3,\3,\2
^(.*),(.*),(.*),\3,\3,\1
^(.*),(.*),(.*),\3,\2,\3
^(.*),(.*),(.*),\3,\2,\2
^(.*),(.*),(.*),\3,\2,\1
^(.*),(.*),(.*),\3,\1,\3
^(.*),(.*),(.*),\3,\1,\2
^(.*),(.*),(.*),\3,\1,\1
^(.*),(.*),(.*),\2,\3,\3
^(.*),(.*),(.*),\2,\3,\2
^(.*),(.*),(.*),\2,\3,\1
^(.*),(.*),(.*),\2,\2,\3
^(.*),(.*),(.*),\2,\2,\2
^(.*),(.*),(.*),\2,\2,\1
^(.*),(.*),(.*),\2,\1,\3
^(.*),(.*),(.*),\2,\1,\2
^(.*),(.*),(.*),\2,\1,\1
^(.*),(.*),(.*),\1,\3,\3
^(.*),(.*),(.*),\1,\3,\2
^(.*),(.*),(.*),\1,\3,\1
^(.*),(.*),(.*),\1,\2,\3
^(.*),(.*),(.*),\1,\2,\2
^(.*),(.*),(.*),\1,\2,\1
^(.*),(.*),(.*),\1,\1,\3
^(.*),(.*),(.*),\1,\1,\2
^(.*),(.*),(.*),\1,\1,\1
^(.*),(.*),(.*),\3,\3,\3,\3
^(.*),(.*),(.*

^(.*),(.*),(.*),\1,\2,\3,\1,\1
^(.*),(.*),(.*),\1,\2,\2,\3,\3
^(.*),(.*),(.*),\1,\2,\2,\3,\2
^(.*),(.*),(.*),\1,\2,\2,\3,\1
^(.*),(.*),(.*),\1,\2,\2,\2,\3
^(.*),(.*),(.*),\1,\2,\2,\2,\2
^(.*),(.*),(.*),\1,\2,\2,\2,\1
^(.*),(.*),(.*),\1,\2,\2,\1,\3
^(.*),(.*),(.*),\1,\2,\2,\1,\2
^(.*),(.*),(.*),\1,\2,\2,\1,\1
^(.*),(.*),(.*),\1,\2,\1,\3,\3
^(.*),(.*),(.*),\1,\2,\1,\3,\2
^(.*),(.*),(.*),\1,\2,\1,\3,\1
^(.*),(.*),(.*),\1,\2,\1,\2,\3
^(.*),(.*),(.*),\1,\2,\1,\2,\2
^(.*),(.*),(.*),\1,\2,\1,\2,\1
^(.*),(.*),(.*),\1,\2,\1,\1,\3
^(.*),(.*),(.*),\1,\2,\1,\1,\2
^(.*),(.*),(.*),\1,\2,\1,\1,\1
^(.*),(.*),(.*),\1,\1,\3,\3,\3
^(.*),(.*),(.*),\1,\1,\3,\3,\2
^(.*),(.*),(.*),\1,\1,\3,\3,\1
^(.*),(.*),(.*),\1,\1,\3,\2,\3
^(.*),(.*),(.*),\1,\1,\3,\2,\2
^(.*),(.*),(.*),\1,\1,\3,\2,\1
^(.*),(.*),(.*),\1,\1,\3,\1,\3
^(.*),(.*),(.*),\1,\1,\3,\1,\2
^(.*),(.*),(.*),\1,\1,\3,\1,\1
^(.*),(.*),(.*),\1,\1,\2,\3,\3
^(.*),(.*),(.*),\1,\1,\2,\3,\2
^(.*),(.*),(.*),\1,\1,\2,\3,\1
^(.*),(.*),(.*),\1,\1,\2,\2,\3
^(.*),(.

^(.*),(.*),(.*),\2,\3,\2,\1,\1,\1
^(.*),(.*),(.*),\2,\3,\1,\3,\3,\3
^(.*),(.*),(.*),\2,\3,\1,\3,\3,\2
^(.*),(.*),(.*),\2,\3,\1,\3,\3,\1
^(.*),(.*),(.*),\2,\3,\1,\3,\2,\3
^(.*),(.*),(.*),\2,\3,\1,\3,\2,\2
^(.*),(.*),(.*),\2,\3,\1,\3,\2,\1
^(.*),(.*),(.*),\2,\3,\1,\3,\1,\3
^(.*),(.*),(.*),\2,\3,\1,\3,\1,\2
^(.*),(.*),(.*),\2,\3,\1,\3,\1,\1
^(.*),(.*),(.*),\2,\3,\1,\2,\3,\3
^(.*),(.*),(.*),\2,\3,\1,\2,\3,\2
^(.*),(.*),(.*),\2,\3,\1,\2,\3,\1
^(.*),(.*),(.*),\2,\3,\1,\2,\2,\3
^(.*),(.*),(.*),\2,\3,\1,\2,\2,\2
^(.*),(.*),(.*),\2,\3,\1,\2,\2,\1
^(.*),(.*),(.*),\2,\3,\1,\2,\1,\3
^(.*),(.*),(.*),\2,\3,\1,\2,\1,\2
^(.*),(.*),(.*),\2,\3,\1,\2,\1,\1
^(.*),(.*),(.*),\2,\3,\1,\1,\3,\3
^(.*),(.*),(.*),\2,\3,\1,\1,\3,\2
^(.*),(.*),(.*),\2,\3,\1,\1,\3,\1
^(.*),(.*),(.*),\2,\3,\1,\1,\2,\3
^(.*),(.*),(.*),\2,\3,\1,\1,\2,\2
^(.*),(.*),(.*),\2,\3,\1,\1,\2,\1
^(.*),(.*),(.*),\2,\3,\1,\1,\1,\3
^(.*),(.*),(.*),\2,\3,\1,\1,\1,\2
^(.*),(.*),(.*),\2,\3,\1,\1,\1,\1
^(.*),(.*),(.*),\2,\2,\3,\3,\3,\3
^(.*),(.*),(.*

^(.*),(.*),(.*),\1,\1,\3,\3,\2,\3
^(.*),(.*),(.*),\1,\1,\3,\3,\2,\2
^(.*),(.*),(.*),\1,\1,\3,\3,\2,\1
^(.*),(.*),(.*),\1,\1,\3,\3,\1,\3
^(.*),(.*),(.*),\1,\1,\3,\3,\1,\2
^(.*),(.*),(.*),\1,\1,\3,\3,\1,\1
^(.*),(.*),(.*),\1,\1,\3,\2,\3,\3
^(.*),(.*),(.*),\1,\1,\3,\2,\3,\2
^(.*),(.*),(.*),\1,\1,\3,\2,\3,\1
^(.*),(.*),(.*),\1,\1,\3,\2,\2,\3
^(.*),(.*),(.*),\1,\1,\3,\2,\2,\2
^(.*),(.*),(.*),\1,\1,\3,\2,\2,\1
^(.*),(.*),(.*),\1,\1,\3,\2,\1,\3
^(.*),(.*),(.*),\1,\1,\3,\2,\1,\2
^(.*),(.*),(.*),\1,\1,\3,\2,\1,\1
^(.*),(.*),(.*),\1,\1,\3,\1,\3,\3
^(.*),(.*),(.*),\1,\1,\3,\1,\3,\2
^(.*),(.*),(.*),\1,\1,\3,\1,\3,\1
^(.*),(.*),(.*),\1,\1,\3,\1,\2,\3
^(.*),(.*),(.*),\1,\1,\3,\1,\2,\2
^(.*),(.*),(.*),\1,\1,\3,\1,\2,\1
^(.*),(.*),(.*),\1,\1,\3,\1,\1,\3
^(.*),(.*),(.*),\1,\1,\3,\1,\1,\2
^(.*),(.*),(.*),\1,\1,\3,\1,\1,\1
^(.*),(.*),(.*),\1,\1,\2,\3,\3,\3
^(.*),(.*),(.*),\1,\1,\2,\3,\3,\2
^(.*),(.*),(.*),\1,\1,\2,\3,\3,\1
^(.*),(.*),(.*),\1,\1,\2,\3,\2,\3
^(.*),(.*),(.*),\1,\1,\2,\3,\2,\2
^(.*),(.*),(.*

^(.*),(.*),(.*),\3,\3,\1,\2,\2,\2,\3
^(.*),(.*),(.*),\3,\3,\1,\2,\2,\2,\2
^(.*),(.*),(.*),\3,\3,\1,\2,\2,\2,\1
^(.*),(.*),(.*),\3,\3,\1,\2,\2,\1,\3
^(.*),(.*),(.*),\3,\3,\1,\2,\2,\1,\2
^(.*),(.*),(.*),\3,\3,\1,\2,\2,\1,\1
^(.*),(.*),(.*),\3,\3,\1,\2,\1,\3,\3
^(.*),(.*),(.*),\3,\3,\1,\2,\1,\3,\2
^(.*),(.*),(.*),\3,\3,\1,\2,\1,\3,\1
^(.*),(.*),(.*),\3,\3,\1,\2,\1,\2,\3
^(.*),(.*),(.*),\3,\3,\1,\2,\1,\2,\2
^(.*),(.*),(.*),\3,\3,\1,\2,\1,\2,\1
^(.*),(.*),(.*),\3,\3,\1,\2,\1,\1,\3
^(.*),(.*),(.*),\3,\3,\1,\2,\1,\1,\2
^(.*),(.*),(.*),\3,\3,\1,\2,\1,\1,\1
^(.*),(.*),(.*),\3,\3,\1,\1,\3,\3,\3
^(.*),(.*),(.*),\3,\3,\1,\1,\3,\3,\2
^(.*),(.*),(.*),\3,\3,\1,\1,\3,\3,\1
^(.*),(.*),(.*),\3,\3,\1,\1,\3,\2,\3
^(.*),(.*),(.*),\3,\3,\1,\1,\3,\2,\2
^(.*),(.*),(.*),\3,\3,\1,\1,\3,\2,\1
^(.*),(.*),(.*),\3,\3,\1,\1,\3,\1,\3
^(.*),(.*),(.*),\3,\3,\1,\1,\3,\1,\2
^(.*),(.*),(.*),\3,\3,\1,\1,\3,\1,\1
^(.*),(.*),(.*),\3,\3,\1,\1,\2,\3,\3
^(.*),(.*),(.*),\3,\3,\1,\1,\2,\3,\2
^(.*),(.*),(.*),\3,\3,\1,\1,\2,\3,\1
^

...

^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\2,\2,\3
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\2,\2,\2
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\2,\2,\1
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\2,\1,\3
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\2,\1,\2
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\2,\1,\1
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\1,\3,\3
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\1,\3,\2
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\1,\3,\1
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\1,\2,\3
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\1,\2,\2
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\1,\2,\1
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\1,\1,\3
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\1,\1,\2
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\3,\1,\1,\1
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\2,\3,\3,\3
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\2,\3,\3,\2
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\2,\3,\3,\1
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\2,\3,\2,\3
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\2,\3,\2,\2
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\2,\3,\2,\1
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\2,\3,\1,\3
^(.*),(.*),(.*),\3,\2,\3,\1,\1,\2,\3,\1,\2
^(.*),(.*),

^(.*),(.*),(.*),\3,\2,\2,\3,\2,\1,\1,\1,\3
^(.*),(.*),(.*),\3,\2,\2,\3,\2,\1,\1,\1,\2
^(.*),(.*),(.*),\3,\2,\2,\3,\2,\1,\1,\1,\1
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\3,\3,\3
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\3,\3,\2
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\3,\3,\1
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\3,\2,\3
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\3,\2,\2
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\3,\2,\1
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\3,\1,\3
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\3,\1,\2
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\3,\1,\1
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\2,\3,\3
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\2,\3,\2
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\2,\3,\1
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\2,\2,\3
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\2,\2,\2
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\2,\2,\1
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\2,\1,\3
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\2,\1,\2
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\2,\1,\1
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\1,\3,\3
^(.*),(.*),(.*),\3,\2,\2,\3,\1,\3,\1,\3,\2
^(.*),(.*),

^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\2,\3,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\2,\2,\3
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\2,\2,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\2,\2,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\2,\1,\3
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\2,\1,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\2,\1,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\1,\3,\3
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\1,\3,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\1,\3,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\1,\2,\3
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\1,\2,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\1,\2,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\1,\1,\3
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\1,\1,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\2,\1,\1,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\1,\3,\3,\3
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\1,\3,\3,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\1,\3,\3,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\1,\3,\2,\3
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\1,\3,\2,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\1,\3,\2,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\2,\1,\3,\1,\3
^(.*),(.*),

Excessive output truncated after 524330 bytes.

^(.*),(.*),(.*),\3,\2,\2,\2,\1,\2,\1,\1,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\1,\2,\1,\1,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\1,\1,\3,\3,\3
^(.*),(.*),(.*),\3,\2,\2,\2,\1,\1,\3,\3,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\1,\1,\3,\3,\1
^(.*),(.*),(.*),\3,\2,\2,\2,\1,\1,\3,\2,\3
^(.*),(.*),(.*),\3,\2,\2,\2,\1,\1,\3,\2,\2
^(.*),(.*),(.*),\3,\2,\2,\2,\1,\1,\3,\2,\1


In [57]:
s = "R,6,L,12,R,6,R,6,L,12,R,6,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,6,L,12,R,6"
m1 = match(r"^(.*),.*\1.*$", s)

RegexMatch("R,6,L,12,R,6,R,6,L,12,R,6,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,6,L,12,R,6", 1="R,6,L,12,R,6")

In [58]:
s2 = replace(s, m1[1] => "A")[5:end]

"L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,A"

In [59]:
m2 = match(r"^(.*),.*\1.*$", s2)

RegexMatch("L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,A", 1="L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12")

In [61]:
res = replace(replace(replace(s, m1[1] => "A"), "L,12,R,6,L,8,L,12" => "B"), "R,12,L,10,L,10" => "C")

"A,A,B,C,B,C,B,C,B,A"

In [91]:
res = "A,A,B,C,B,C,B,C,B,A"
a = "R,6,L,12,R,6"
b = "L,12,R,6,L,8,L,12"
c = "R,12,L,10,L,10"

"R,12,L,10,L,10"

In [92]:
res1 = [Int.(collect(res)); 10]
a1 = [Int.(collect(a)); 10]
b1 = [Int.(collect(b)); 10]
c1 = [Int.(collect(c)); 10]

15-element Array{Int64,1}:
 82
 44
 49
 50
 44
 76
 44
 49
 48
 44
 76
 44
 49
 48
 10

In [95]:
res = "A,A,B,C,B,C,B,C,B,A"
a = "R,6,L,12,R,6"
b = "L,12,R,6,L,8,L,12"
c = "R,12,L,10,L,10"

res1 = [Int.(collect(res)); 10]
a1 = [Int.(collect(a)); 10]
b1 = [Int.(collect(b)); 10]
c1 = [Int.(collect(c)); 10]

code = collect(readline("input.txt"))
code[1] = '2'
code = join(code)
vm = Prog(code)
out = run(vm)
print(join(Char.(out)))
out = run(vm, res1)
print(join(Char.(out)))
out = run(vm, a1)
print(join(Char.(out)))
out = run(vm, b1)
print(join(Char.(out)))
out = run(vm, c1)
print(join(Char.(out)))
out = run(vm, [Int('n'), 10])
print(join(Char.(out)))

..............................................###########......
..............................................#.........#......
..............................................#.........#......
..............................................#.........#......
......#######.................................#.........#......
......#.....#.................................#.........#......
......#.....#.........#.......................#.........#......
......#.....#.........#.......................#.........#......
......#.....#.........#.......................#.........#......
......#.....#.........#.......................#.........#......
......#.....#############...................#############......
......#...............#.#...................#.#................
......#...............#############.....#######................
......#.................#.........#.....#...#..................
......#.................#.........#.....#...#..................
......#.................#.........#.....

In [96]:
out[end]

1063081

In [80]:
out = run(vm, [Int('n'), 10])

0-element Array{Int64,1}

In [89]:
out = run(vm, 82)
print(join(Char.(out)))


Expected function name but got: R


In [93]:
res1

20-element Array{Int64,1}:
 65
 44
 65
 44
 66
 44
 67
 44
 66
 44
 67
 44
 66
 44
 67
 44
 66
 44
 65
 10

In [173]:
print(join(Char.(out)))match(r"^(.*),(.*),(.*),\2,\1,\3$", "R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2")

RegexMatch("R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2", 1="R,8,R,8", 2="R,4,R,4,R,8", 3="L,6,L,2")

In [52]:
s = "R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2"
m1 = match(r"^(.*),.*\1.*$", s)
#s2 = replace(s, m1[1] => "A")
#s2 = replace(split(s2, "A")[2], r"^,|,$" => "")

RegexMatch("R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2", 1="R,8,R,8,R")

In [50]:
m2 = match(r"^(.*),.*\1.*$", s2)

RegexMatch("4,R,4,R,8,L,6,L,2,R,4,R,4", 1="4,R,4")

In [44]:
m = match(r"^(.*),(.*),(.*)$", "R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2")

RegexMatch("R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2", 1="R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6", 2="L", 3="2")

In [196]:
all(length.(m.captures) .<= 20)

false

In [190]:
Regex("abc")

r"abc"

In [147]:
match(r"(.*)(.*)(.*)\1\3\2", "ABCABC")

RegexMatch("ABCABC", 1="ABC", 2="", 3="")

In [97]:
replace("R,6,L,12,R,6,R,6,L,12,R,6,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,6,L,12,R,6", "R,6" => "x1")

"x1,L,12,x1,x1,L,12,x1,L,12,x1,L,8,L,12,R,12,L,10,L,10,L,12,x1,L,8,L,12,R,12,L,10,L,10,L,12,x1,L,8,L,12,R,12,L,10,L,10,L,12,x1,L,8,L,12,x1,L,12,x1"

In [100]:
m = match(r"^(.{1,21})\1*(.{1,21})(?:\1|\2)*(.{1,21})(?:\1|\2|\3)*$", "R,6,L,12,R,6,R,6,L,12,R,6,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,6,L,12,R,6")

In [117]:
s = "R,6,L,12,R,6,R,6,L,12,R,6,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,12,L,10,L,10,L,12,R,6,L,8,L,12,R,6,L,12,R,6"
s = "R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2"
m = match(r"^(.{1,21})\1*(.{1,21})(?:\1|\2)*(.{1,21})(?:\1|\2|\3)*$", s)

RegexMatch("R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2", 1="R,8,R,8,R,4,R,4,R,8,L", 2=",6,L,2,R,4,R,4,R,8,R,", 3="8,R,8,L,6,L,2")

In [119]:
replace(replace(replace(s, m[1] => "A"), m[2] => "B"), m[3] => "C")

"ABC"

In [116]:
match(r"^(.)(.)(?:\1|\2)*$", "xyxyyyx")

RegexMatch("xyxyyyx", 1="x", 2="y")